# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
# your code here
import pandas as pd
import numpy as np

tic = pd.read_csv('tic-tac-toe.csv')

In [2]:
print(tic.shape)
print(tic.dtypes)
tic.head()

(958, 10)
TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class      bool
dtype: object


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [3]:
null_cols = tic.isna().sum()
null_cols # I figured, but still.

TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

In [4]:
encoding = {"x":  1, "o": 2, "b":0, True:1, False:0}
tic.replace(encoding, inplace=True)
tic.head()
tic.dtypes

TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class     int64
dtype: object

In [6]:
arr = tic.values
arr

x = arr[:, :-1]
y = arr[:, -1:]
y = y.ravel()
x

array([[1, 1, 1, ..., 1, 2, 2],
       [1, 1, 1, ..., 2, 1, 2],
       [1, 1, 1, ..., 2, 2, 1],
       ...,
       [2, 1, 2, ..., 1, 2, 1],
       [2, 1, 2, ..., 1, 2, 1],
       [2, 2, 1, ..., 2, 1, 1]], dtype=object)

In [8]:
x = x.astype(float)
y = y.astype(float)

In [9]:
from sklearn.preprocessing import normalize

x = normalize(x, axis=0, norm='max')

In [10]:
print(x.shape)
print(y.shape)

(958, 9)
(958,)


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [11]:
# your code here
import tensorflow as tf

tf.__version__

'2.1.0'

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(766, 9)
(766,)
(192, 9)
(192,)


In [14]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(9, activation=tf.nn.softmax))

In [15]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train,y_train, epochs=3)

Train on 766 samples
Epoch 1/3
766/766 [==============================] - 1s 663us/sample - loss: 1.3055 - accuracy: 0.5836
Epoch 2/3
766/766 [==============================] - 0s 108us/sample - loss: 0.6640 - accuracy: 0.6540
Epoch 3/3
766/766 [==============================] - 0s 143us/sample - loss: 0.6220 - accuracy: 0.6540


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [17]:
val_loss, val_acc = model.evaluate(X_test,y_test)
print(val_loss, val_acc)

model.save('tic-tac-toe.model')

192/192 [==============================] - 0s 89us/sample - loss: 0.6234 - accuracy: 0.6458
0.6234121123949686 0.6458333
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: tic-tac-toe.model/assets


In [33]:
# your code here
new_model = tf.keras.models.load_model('tic-tac-toe.model')

predictions = new_model.predict(X_test)

pred_loc = np.random.randint(len(X_test), size=5)
print(pred_loc)

for i in pred_loc:
    prediction = []
    for i in pred_loc:
        pred = np.argmax(predictions[i])
        prediction.append(pred)

prediction

[ 99  10  39 186  61]


[1, 1, 1, 1, 1]

In [35]:
print(y_test[99])
print(y_test[10])
print(y_test[39])
print(y_test[186])
print(y_test[61])
# so it got three out of five right.

1.0
1.0
0.0
0.0
1.0


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(9, activation=tf.nn.softmax))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train,y_train, epochs=3)

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here